<a href="https://colab.research.google.com/github/fuchuanzhe/MachineVision/blob/main/Machine_Vision_Final_Lab_Model_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== INSTALL DEPENDENCIES =====
!pip install huggingface_hub
!pip install boto3 -q
!pip install opencv-python torch numpy torchvision tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 140.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from huggingface_hub import hf_hub_download
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
import cv2
import numpy as np
from tqdm import tqdm
import time

# Please double, triple, quadruple check that the below code runs without errors before submitting.

## TODO 1 - Enter your HuggingFace username below:

In [ ]:
hf_username = "rossamurphy"

## TODO 2 - Define your model EXACTLY as you did in your training code (otherwise there will be errors, and, possibly, tears).

Note below the classname is 'YourModelArchitecture'. That's because it literally needs to be YOUR MODEL ARCHITECTURE. This class definition is later referred to below in the 'load_model_from_hub' method. The architecture must match here, or it will not be able to instantiate the model weights correctly once it downloads them from HuggingFace. Pay very close attention to getting this right, please.

Replace the below code, and replace the corresponding line in the 'load_model_from_hub' method.

In [ ]:
# =============================================================================
# 1. MODEL DEFINITION (must match training)
# =============================================================================

class YourModelArchitecture(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((4, 4))
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = x.mean(dim=2)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


## Download the test data from s3, and create the corresponding dataset + dataloader.

There's no TODO for you here. This text is just here to explain to you what this code does.

In this instance, the test data IS the training data you were provided in the Model Training notebook. This is by design. You do not have access to the test data. This is a simple check to make sure the mechanics of this notebook work.

You should achieve the same accuracy here in this notebook, as you did in your previous notebook (random seed notwithstanding).

In [ ]:
# =============================================================================
# DOWNLOAD TEST DATA FROM S3
# =============================================================================

def download_test_data(bucket_name='training-and-validation-data',download_dir='./test-data'):
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    bucket_name = 'prism-mvta'
    prefix = 'training-and-validation-data/'

    os.makedirs(download_dir, exist_ok=True)

    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    video_names = []

    for page in pages:
        if 'Contents' not in page:
            print("No files found at the specified path!")
            break

        print("Downloading test data:\n")
        for obj in tqdm(page['Contents']):
            key = obj['Key']
            filename = os.path.basename(key)

            if not filename:
                continue

            video_names.append(filename)
            local_path = os.path.join(download_dir, filename)
            # print(f"Downloading: {filename}")
            s3.download_file(bucket_name, key, local_path)

    print(f"\nDownloaded {len(video_names)} test videos")
    return download_dir


# ============================================================================= # DATASET AND DATALOADER =============================================================================

class VideoDataset(Dataset):
    def __init__(self, video_dir, frame_size=(224, 224)):
        self.video_dir = video_dir
        self.frame_size = frame_size

        self.video_files = [
            f for f in os.listdir(video_dir)
            if f.endswith(('.mp4', '.avi', '.mov'))
        ]

        self.labels = [
            int(f.split('_')[0]) for f in self.video_files
        ]

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = os.path.join(self.video_dir, self.video_files[idx])
        frames = self._load_video(video_path)
        label = self.labels[idx]
        return frames, label

    def _load_video(self, path):
        cap = cv2.VideoCapture(path)

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (self.frame_size[1], self.frame_size[0]))
            frames.append(frame)

        cap.release()

        frames = torch.from_numpy(np.array(frames)).permute(3, 0, 1, 2).float() / 255.0
        return frames


def collate_fn(batch):
    """Pad all videos to 1000 frames."""
    frames_list, labels = zip(*batch)

    target_frames = 1000

    padded_frames = []
    for frames in frames_list:
        num_frames = frames.shape[1]
        if num_frames < target_frames:
            padding = torch.zeros(frames.shape[0], target_frames - num_frames, frames.shape[2], frames.shape[3])
            frames = torch.cat([frames, padding], dim=1)
        elif num_frames > target_frames:
            frames = frames[:, :target_frames, :, :]
        padded_frames.append(frames)

    frames_batch = torch.stack(padded_frames, dim=0)
    labels_batch = torch.tensor(labels)

    return frames_batch, labels_batch


## TODO 3 - Download your model from HuggingFace and instantiate it

Replace line 8 of the below code. Line 8 is where you instantiate YOUR MODEL ARCHITECTURE (which you re-defined above) with the weights you download from HuggingFace. Make sure you get the class name, and the arguments to the __init__ method correct.


This code just downloads the same model which you uploaded in the last notebook.

In [ ]:
# =============================================================================
# DOWNLOAD MODEL FROM HUGGING FACE
# =============================================================================

def load_model_from_hub(repo_id, num_classes=10):
    model_path = hf_hub_download(repo_id=repo_id, filename="model.pt")

    model = YourModelArchitecture(num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location='cpu'))

    print(f"Model loaded from {repo_id}")
    return model

model = load_model_from_hub(f"{hf_username}/mv-final-assignment",num_classes=10)



model.pt:   0%|          | 0.00/611k [00:00<?, ?B/s]

Model loaded from rossamurphy/mv-final-assignment


## TODO 4

Make sure the below code correctly evaluates your model performance on the given data!

This is your last chance to verify this before submission.

In [ ]:
def evaluate(model, test_loader, dataset, device):
    model.eval()
    correct = 0
    total = 0

    all_preds = []
    all_labels = []
    all_times = []

    print("\n")

    with torch.no_grad():
        for idx, (frames, labels) in enumerate(test_loader):
            frames, labels = frames.to(device), labels.to(device)

            # Time the forward pass
            start_time = time.time()
            outputs = model(frames)
            if device.type == 'cuda':
                torch.cuda.synchronize()  # wait for GPU to finish
            end_time = time.time()

            inference_time = (end_time - start_time) * 1000  # ms
            all_times.append(inference_time)

            preds = outputs.argmax(dim=1)

            for i in range(labels.size(0)):
                batch_idx = idx * test_loader.batch_size + i
                video_name = dataset.video_files[batch_idx]
                pred = preds[i].item()
                true_label = labels[i].item()
                is_correct = "✓" if pred == true_label else "✗"

                print(f"{is_correct}  pred={pred}  true={true_label}  |  {inference_time:>7.1f}ms  |  {video_name}")

            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    return accuracy, all_preds, all_labels, all_times


# =============================================================================
# RUN INFERENCE
# =============================================================================

def run_inference(model, bucket_name='training-and-validation-data'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Download test data
    test_dir = download_test_data(bucket_name, './test-data')

    model = model.to(device)

    # Create dataloader
    test_dataset = VideoDataset(test_dir, frame_size=(224, 224))
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        collate_fn=collate_fn
    )

    print(f"\nRunning inference on {len(test_dataset)} test videos...")

    # Warmup (optional, helps get consistent GPU timings)
    if device.type == 'cuda':
        dummy = torch.randn(1, 3, 1000, 224, 224).to(device)
        with torch.no_grad():
            _ = model(dummy)
        torch.cuda.synchronize()

    total_start = time.time()
    accuracy, preds, labels, times = evaluate(model, test_loader, test_dataset, device)
    total_end = time.time()

    # Summary
    num_correct = sum(p == l for p, l in zip(preds, labels))
    num_wrong = len(preds) - num_correct

    print("\n" + "="*50)
    print("SUMMARY")
    print("="*50)
    print(f"Total videos:         {len(preds)}")
    print(f"Correct:              {num_correct}")
    print(f"Incorrect:                {num_wrong}")
    print(f"")
    print(f"ACCURACY:             {accuracy*100:.2f}%")
    print(f"")
    print(f"Total time:           {total_end - total_start:.2f}s")
    print(f"Avg per video:        {sum(times) / len(times):.1f}ms")
    print(f"Min latency:          {min(times):.1f}ms")
    print(f"Max latency:          {max(times):.1f}ms")
    print("="*50)
    return accuracy, preds, labels

_, _, _ = run_inference(model)

Using device: cuda



100%|██████████| 77/77 [00:53<00:00,  1.44it/s]



Downloaded 77 test videos

Running inference on 77 test videos...


✗  pred=3  true=4  |      2.9ms  |  4_kling_20251209_Text_to_Video_Generate_a_452_1.mp4
✓  pred=3  true=3  |      2.7ms  |  3_kling_20251206_Text_to_Video_Generate_a_71_2.mp4
✓  pred=3  true=3  |      2.7ms  |  3_sadlfkjasldkfjasleijlkjfd.mp4
✓  pred=3  true=3  |      2.7ms  |  3_kling_20251209_Text_to_Video_Generate_a_491_0.mp4
✓  pred=3  true=3  |      2.7ms  |  3_kling_20251209_Image_to_Video_Generate_a_635_0.mp4
✗  pred=3  true=4  |      2.7ms  |  4_kling_20251209_Image_to_Video_Generate_a_635_1.mp4
✗  pred=3  true=2  |      2.7ms  |  2_kling_20251205_Text_to_Video_On_a_sandy_4976_0.mp4
✗  pred=3  true=4  |      2.7ms  |  4_20251209_Text_to_Video_Generate_a_561_0.mp4
✓  pred=3  true=3  |      2.7ms  |  3_dslkaldskjflakjs.mp4
✗  pred=3  true=4  |      2.7ms  |  4_kling_20251206_Text_to_Video_Generate_a_28_0.mp4
✓  pred=3  true=3  |      2.7ms  |  3_sdlkjslndflkseijlkjef.mp4
✗  pred=3  true=2  |      2.7ms  |  2_sdl